**Importing Libraries**
--






In [1]:
import pandas as pd
import numpy as np
import nltk  
import random  
import string 
from sklearn.base import TransformerMixin

**Reading Data From CSV File**
--

In [2]:
data = pd.read_csv('flipkart.csv', low_memory=False)

**Printing DataFrame**
--

In [3]:
display(data)

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,7179d2f6c4ad50a17d014ca1d2815156,2015-12-01 10:15:43 +0000,http://www.flipkart.com/walldesign-small-vinyl...,WallDesign Small Vinyl Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7KFJAKSTDY9G,1500.0,730.0,"[""http://img6a.flixcart.com/image/wall-decorat...",False,Buy WallDesign Small Vinyl Sticker for Rs.730 ...,No rating available,No rating available,WallDesign,"{""product_specification""=>[{""key""=>""Number of ..."
19996,71ac419198359d37b8fe5e3fffdfee09,2015-12-01 10:15:43 +0000,http://www.flipkart.com/wallmantra-large-vinyl...,Wallmantra Large Vinyl Stickers Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE9F5URNQGJCGH,1429.0,1143.0,"[""http://img6a.flixcart.com/image/sticker/z/g/...",False,Buy Wallmantra Large Vinyl Stickers Sticker fo...,No rating available,No rating available,Wallmantra,"{""product_specification""=>[{""key""=>""Number of ..."
19997,93e9d343837400ce0d7980874ece471c,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7VAYDKQZEBSD,1299.0,999.0,"[""http://img5a.flixcart.com/image/sticker/b/s/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ..."
19998,669e79b8fa5d9ae020841c0c97d5e935,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,Elite Collection Medium Acrylic Sticker,"[""Baby Care >>

**Cleaning Messy Column Names**
--

In [4]:
data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
data.columns

Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_fk_advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object')

**Creating a NEW dataframe from acutal dataframe by selecting specfic columns**
--

In [5]:
df = data[['brand','description']]

**Displaying the NEW dataframe**
--

In [6]:
display(df)

,brand,description
0,Alisha,Key Features of Alisha Solid Women's Cycling S...
1,FabHomeDecor,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,AW,Key Features of AW Bellies Sandals Wedges Heel...
3,Alisha,Key Features of Alisha Solid Women's Cycling S...
4,Sicons,Specifications of Sicons All Purpose Arnica Do...
...,...,...
19995,WallDesign,Buy WallDesign Small Vinyl Sticker for Rs.730 ...
19996,Wallmantra,Buy Wallmantra Large Vinyl Stickers Sticker fo...
19997,Elite Collection,Buy Elite Collection Medium Acrylic Sticker fo...
19998,Elite Collection,Buy Elite Collection Medium Acrylic Sticker fo...


**Removing Special Characters from Dataframe**
--

In [7]:
# Replacing dataFrame in another Variable.
df_spsl = df

# Declaration all Sorts of Special Characters in the form of List.
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–", "//", "%*", ":/", ".;", "Ø", "§"]

# Checking
for c in spec_chars:
    c1 = "\\" + c
    # Replacing With SPACE"".
    df_spsl=df_spsl.replace(c1,"", regex=True)

# Replacing DataFrame with no Special characters.
df1 = df_spsl
display(df1)

,brand,description
0,Alisha,Key Features of Alisha Solid Womens Cycling Sh...
1,FabHomeDecor,FabHomeDecor Fabric Double Sofa Bed Finish Col...
2,AW,Key Features of AW Bellies Sandals Wedges Heel...
3,Alisha,Key Features of Alisha Solid Womens Cycling Sh...
4,Sicons,Specifications of Sicons All Purpose Arnica Do...
...,...,...
19995,WallDesign,Buy WallDesign Small Vinyl Sticker for Rs730 o...
19996,Wallmantra,Buy Wallmantra Large Vinyl Stickers Sticker fo...
19997,Elite Collection,Buy Elite Collection Medium Acrylic Sticker fo...
19998,Elite Collection,Buy Elite Collection Medium Acrylic Sticker fo...


**Counting NAN values in dataframe**
--

In [8]:
df1.isna().sum()

brand          5864
description       2
dtype: int64

**Creating a custom class to Impute Null values using TransformerMixin**
--

In [9]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

**Applying Imputation on a Dataframe**
--

In [10]:
df = DataFrameImputer().fit_transform(df1)
display(df)

,brand,description
0,Alisha,Key Features of Alisha Solid Womens Cycling Sh...
1,FabHomeDecor,FabHomeDecor Fabric Double Sofa Bed Finish Col...
2,AW,Key Features of AW Bellies Sandals Wedges Heel...
3,Alisha,Key Features of Alisha Solid Womens Cycling Sh...
4,Sicons,Specifications of Sicons All Purpose Arnica Do...
...,...,...
19995,WallDesign,Buy WallDesign Small Vinyl Sticker for Rs730 o...
19996,Wallmantra,Buy Wallmantra Large Vinyl Stickers Sticker fo...
19997,Elite Collection,Buy Elite Collection Medium Acrylic Sticker fo...
19998,Elite Collection,Buy Elite Collection Medium Acrylic Sticker fo...


**Checking for NA values again**
--

In [11]:
df.isna().sum()
df.shape[0]

20000

In [12]:
# Cleaning the texts
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
rows = df.shape[0]
for i in range(0, rows):
    des = re.sub('[^a-zA-Z]', ' ', df['description'][i])
    des = des.lower()
    des = des.split()
    ps = PorterStemmer()
    des = [ps.stem(word) for word in des if not word in set(stopwords.words('english'))]
    des = ' '.join(des)
    corpus.append(des)

# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 10)
X = cv.fit_transform(corpus).toarray()
X

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


array([[0, 0, 0, ..., 0, 0, 3],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 0, 1, 1],
       ...,
       [1, 1, 0, ..., 1, 1, 0],
       [1, 1, 0, ..., 1, 1, 0],
       [1, 1, 0, ..., 1, 1, 0]], dtype=int64)

** With Keras **